In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd

# CSV dosyasını yükle
csv_path = './montgomery_metadata.csv'
data_info = pd.read_csv(csv_path)

# Veri yolları
image_dir = './images/images'  # Görüntüler klasörü
mask_dir = './images/masks'    # Maskeler klasörü
image_size = (256, 256)  # Resimler ve maskeler için sabit boyut

# 1. Veri Hazırlığı
# CSV dosyasını kullanarak veri yükleme

def load_data_with_csv(data_info, image_dir, mask_dir, image_size):
    images = []
    masks = []

    for _, row in data_info.iterrows():
        # CSV'den görüntü ve maske isimlerini al
        filename = row['study_id']

        img_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename)

        if os.path.exists(img_path) and os.path.exists(mask_path):
            # Görüntüyü ve maskeyi yükle
            img = load_img(img_path, target_size=image_size)
            mask = load_img(mask_path, target_size=image_size, color_mode='grayscale')

            # Normalize et (0-1 arası)
            images.append(img_to_array(img) / 255.0)
            masks.append(img_to_array(mask) / 255.0)

    return np.array(images), np.array(masks)

# Veriyi yükle
images, masks = load_data_with_csv(data_info, image_dir, mask_dir, image_size)

# Veriyi eğitim ve test setlerine böl
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# 2. U-Net Modeli Tanımlama
def unet_model(input_size):
    inputs = layers.Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u1 = layers.concatenate([u1, c2])
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u2 = layers.concatenate([u2, c1])
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# Modeli derle
model = unet_model((*image_size, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Eğitim
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=16,
    epochs=20
)

# Modeli kaydet
model.save('unet_model_with_csv.h5')


Epoch 1/20


c:\Users\senol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 256, 256, 3))
  warnings.warn(msg)


7/7 ━━━━━━━━━━━━━━━━━━━━ 45s 6s/step - accuracy: 0.6204 - loss: 0.6408 - val_accuracy: 0.6484 - val_loss: 0.5687
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 6s/step - accuracy: 0.6774 - loss: 0.5600 - val_accuracy: 0.7259 - val_loss: 0.4793
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.7580 - loss: 0.4561 - val_accuracy: 0.7968 - val_loss: 0.3962
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.8705 - loss: 0.3080 - val_accuracy: 0.9338 - val_loss: 0.1479
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.9511 - loss: 0.1292 - val_accuracy: 0.9793 - val_loss: 0.0629
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.9754 - loss: 0.0672 - val_accuracy: 0.9830 - val_loss: 0.0522
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.9805 - loss: 0.0562 - val_accuracy: 0.9820 - val_loss: 0.0484
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 6s/step - accuracy: 0.9813 - loss: 0.0511 - val_accuracy: 0.9844 - val_loss: 0.0439
Epoch 9/20
7/7 ━━━━